In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer as si
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier

In [93]:
df = pd.read_csv("/Users/prajualpillai/Desktop/prajual/Personal_git/boosting/train.csv")

In [94]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [95]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [96]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [97]:
set(df["Embarked"])

{'C', 'Q', 'S', nan}

In [98]:
len(set(df["Ticket"])), len(df["Ticket"])

(681, 891)

In [100]:
# Convert categorical values to numeraical
# 1. One-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()

In [101]:
imp_median = si(missing_values=np.nan, strategy="median")
imp_median.fit(df[["Age"]])
df[["Age"]] = imp_median.transform(df[["Age"]])

In [102]:
df2 = df.drop(columns=["Cabin","Name"], axis=1)
df3 = df2.dropna(how="any", axis=0)

In [103]:
def encode(df_3):
    integer_encoded = label_encoder.fit_transform(df_3)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [ ]:
embarked_encoded = encode(df3["Embarked"])
sex_encoded = encode(df3["Sex"])
df3[["em_1","em_2", "em_3"]] = embarked_encoded
df3[["Sex_1","Sex_2"]] = sex_encoded
integer_encoded = label_encoder.fit_transform(df3["Ticket"])
df3["Ticket"] = integer_encoded.reshape(len(integer_encoded),1)

In [106]:
len(set(df3["Ticket"]))

680

In [112]:
independent_values = df3.drop(columns=["Survived", "Sex", "Embarked"], axis=1)
dependent_value = df3[["Survived"]]

In [113]:
independent_values.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
em_1           0
em_2           0
em_3           0
Sex_1          0
Sex_2          0
dtype: int64

In [114]:
independent_values.dtypes

PassengerId      int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Ticket           int64
Fare           float64
em_1           float64
em_2           float64
em_3           float64
Sex_1          float64
Sex_2          float64
dtype: object

In [115]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [116]:
kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index,val_index in kf.split(independent_values):
    X_train,X_val = independent_values.iloc[train_index],independent_values.iloc[val_index],
    y_train,y_val = dependent_value.iloc[train_index],dependent_value.iloc[val_index],

In [117]:
gradient_booster.fit(X_train,y_train)

/opt/anaconda3/envs/amex/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [119]:
print(classification_report(y_val,gradient_booster.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       116
           1       0.76      0.67      0.71        61

    accuracy                           0.81       177
   macro avg       0.80      0.78      0.79       177
weighted avg       0.81      0.81      0.81       177



In [120]:
import pickle
with open('clf.pickle', 'wb') as f:
    pickle.dump(gradient_booster, f)